<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-up-the-Environment" data-toc-modified-id="Set-up-the-Environment-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set up the Environment</a></span></li><li><span><a href="#Find-the-Critical-Values-Corresponding-to-$\alpha=0.01,-0.05$" data-toc-modified-id="Find-the-Critical-Values-Corresponding-to-$\alpha=0.01,-0.05$-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Find the Critical Values Corresponding to $\alpha=0.01, 0.05$</a></span><ul class="toc-item"><li><span><a href="#Run-Simulations-to-Find-the-Values" data-toc-modified-id="Run-Simulations-to-Find-the-Values-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Run Simulations to Find the Values</a></span></li><li><span><a href="#Check-if-the-Tests'-Sizes-Correspond-to-the-Nominal-Values-of-$\alpha$" data-toc-modified-id="Check-if-the-Tests'-Sizes-Correspond-to-the-Nominal-Values-of-$\alpha$-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Check if the Tests' Sizes Correspond to the Nominal Values of $\alpha$</a></span></li></ul></li></ul></div>

# Robust Tests of Normality <a class="tocSkip">

## Set up the Environment

In [ ]:
import numpy as np
import pandas as pd
import pickle
import robust
import util
import importlib

## Find the Critical Values Corresponding to $\alpha=0.01, 0.05$

### Run Simulations to Find the Values

In [ ]:
# Prepare the normal samples
samples = {}

ns = list(range(2, 101, 1))

for n in ns:
    print(n)
    samples[n] = [np.random.normal(0, 1, n).tolist() for l in range(10000)]

In [ ]:
#critical_values = {}

for test_name in robust.statistics:
    for n in samples:
        if n not in [5]:
            continue
        for alpha in [0.01, 0.05,]:
            print(n, alpha)
            critical_values[test_name][(n, alpha)]  = None

In [ ]:
for name in robust.statistics:
    print(name)
    statistic = robust.statistics[name]
    
    for n in samples:
        #print(f'{n}{"," if n < 100 else ""} ', end='')
        if n != 5:
            continue
            
        print(f'{n}{"," if n < 100 else ""} ', end='')

        scores = [statistic(sample) for sample in samples[n]]
        
        if name in ['MMRT1', 'MMRT2', 'TTRT1', 'TTRT2']:
            thresholds = np.quantile(scores, [0.99, 0.95]).tolist()
        else:
            thresholds = np.quantile(scores, [0.01, 0.05]).tolist()

        for alpha, critical_value in zip([0.01, 0.05], thresholds):
            critical_values[name][(n, alpha)] = critical_value

    print('\n')

In [ ]:
# Save the critical values as a pickle file
with open('robust_critical_values.p', 'wb') as f:
    pickle.dump(critical_values, f)

### Check if the Tests' Sizes Correspond to the Nominal Values of $\alpha$

In [ ]:
samples = {}

for n in range(10, 101, 10):
    samples[n] = [np.random.normal(0, 1, n).tolist() for l in range(10000)]

In [ ]:
tests = {}
for name in ['MMRT1', 'MMRT2', 'TTRT1', 'TTRT2', 'RSW', 'RLM']:
    test, statistic = robust.get_robust_test(name)
    for alpha in [0.01, 0.05]:
        tests[(name, alpha)] = util.TestClassifier(test, statistic, alpha)

In [ ]:
sizes = {}

for n in samples:
    print(n)
    
    sizes[n] = []
    
    for test_name in sorted(tests.keys()):
        rtc = tests[test_name]
        decisions = rtc.predict(samples[n])
      
        size = 1.0 - sum(decisions) / len(decisions)
        sizes[n].append(size)

In [ ]:
df = pd.DataFrame(sizes).T
df.columns = sorted(tests.keys())
df = df.T
df

In the original experiments, it turned out that the empirical $FNR$ could be controlled by setting $\alpha$ to the desired value.